In [0]:
from collections import OrderedDict 
from operator import itemgetter
import urllib
from bs4 import BeautifulSoup
from bs4.element import Comment
from collections import Counter
from string import punctuation
import re



def make_soup(url):
    try:
        page = urllib.request.urlopen(url)
    except urllib.request.HTTPError as e:
        print('Ignored: ', e)
        return -1
        
    req = urllib.request.Request(url , headers={'User-Agent': 'Mozilla/5.0'})
    page = urllib.request.urlopen(req).read()
    soup_data = BeautifulSoup(page, "html.parser")
    return soup_data


def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True


def text_from_html(url):
    soup = make_soup(url)
    if soup == -1:
        return ''
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)  
    return u" ".join(t.strip() for t in visible_texts)


def get_words(url):
    text = text_from_html(url)
    # Delete punctuaction and empty words
    text = [re.sub('\W+', '', t) for t in text.lower().split()]
    text = list(filter(lambda t: len(t), text))
    return text

def get_hrefs(url):
    soup = make_soup(url)
    if soup == -1:
        return []
    hrefs = []
        
    for link in soup.find_all('a'):
        hrefs.append(link.get('href'))
        
    ans = []
    for link in hrefs:
        if link is None or len(link) == 0:
            continue
        if link.find('https') != -1 or link.find('http') != -1:
            ans.append(link)
                      
    return ans

def DFS(url, step, max_step, no, sites_no):
    if step >= max_step:
        return
    
    print(f'{no} / {sites_no} -> : ', url)
        
    words = get_words(url)
    hrefs = get_hrefs(url)
    
    for w in words:
        WORDS.setdefault(w, set()).add(url)
        
    for no, ref in enumerate(hrefs):
        DFS(ref, step + 1, max_step, no + 1, len(hrefs))
        

In [45]:
site = 'https://www.ii.uni.wroc.pl/~marcinm/dyd/python/'
WORDS = {}
steps = 2
#DFS(site, 0, steps, 1, 1)
print(WORDS)

{}


lista 7 wielowątkowość 

In [0]:
import concurrent

In [47]:
def get_multi_dict():
    return dict(sorted(WORDS.items(), key=lambda x: x[1], reverse=True))

def BFS_multi(url, max_steps):
    sites = [url]
    all_sites = [url]
    
    for no in range(max_steps):
        act_sites = []
        # Use ProcessPoolExecutor or ThreadPoolExecutor
        with concurrent.futures.ThreadPoolExecutor() as executor:
            act_sites = list(executor.map(get_hrefs, sites))
        sites = act_sites[0]
        all_sites += act_sites[0]

    sites = all_sites
    words = []

    print('\n Start scrapping \n')
    # Use ProcessPoolExecutor or ThreadPoolExecutor
    with concurrent.futures.ThreadPoolExecutor() as executor:
        words = list(executor.map(get_words, sites))

    for word_list in words:
        for w in word_list:
            WORDS[w] = WORDS.get(w, 0) + 1


    print('\n\n---SITES---\n\n')
    for i, s in enumerate(sites):
        print(f'{i} / {len(sites)} -> : ', s)


def multi_scrap(url, depth):
    WORDS.clear()
    BFS_multi(url, depth)
    return get_multi_dict()

multi_scrap(site, 1)


 Start scrapping 



---SITES---


0 / 8 -> :  https://www.ii.uni.wroc.pl/~marcinm/dyd/python/
1 / 8 -> :  http://www.python.org
2 / 8 -> :  http://www.python.org/
3 / 8 -> :  http://diveintopython.org/
4 / 8 -> :  http://pl.wikipedia.org/wiki/Python
5 / 8 -> :  http://en.wikipedia.org/wiki/Python_%28programming_language%29
6 / 8 -> :  http://pl.python.org
7 / 8 -> :  http://www.tkdocs.com/tutorial/


590